# MultiBand Single Pass (MBSP) Demonstration

This notebook demonstrates a basic implementation of the MBSP algorithm
described in *Varon et al. (2021)* for detecting large methane plumes using
Sentinel‑-2 imagery. The MBSP method compares band‑11 and band‑12
reflectances for a single scene to retrieve methane column enhancements.
We keep core routines in `src/mbsp.py` for reuse, while image selection
and experimentation remain in the notebook.

## MBSP versus MBMP

MBSP uses two spectral bands from a single acquisition. A scaling coefficient
is fitted between bands 11 and 12 over the scene and the fractional absorption
is computed as:

\begin{align}
R_{MBSP} = \frac{c R_{12} - R_{11}}{R_{11}}
\end{align}

where $c$ is the slope from fitting $R_{12}$ to $R_{11}$. This approach relies
on the surface behaving similarly in both bands.

The MultiBand MultiPass (MBMP) technique performs the MBSP retrieval on two
different dates and subtracts them to reduce artifacts. MBMP generally provides
better precision when a good plume‑free reference image is available.

In [ ]:
import datetime as dt

import ee
import geemap

# Authenticate with Earth Engine
ee.Authenticate()
ee.Initialize()


## Helper Functions

In [ ]:
def mask_s2_clouds(image: ee.Image) -> ee.Image:
    """Mask clouds using the QA60 band."""
    qa = image.select("QA60")
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    masked = image.updateMask(mask).divide(10000)
    return masked.copyProperties(image, image.propertyNames())  # <-- keep metadata!

In [ ]:
def mbsp_fractional_image(image: ee.Image, region: ee.Geometry) -> ee.Image:
    """
    Compute the MBSP (Multi-Band Single-Pass) fractional methane signal
    for a Sentinel-2 scene following Varon et al. 2021 (Atmos. Meas. Tech.).

    MBSP definition
    ----------------
        R_MBSP = ( c · R12  –  R11 ) / R11

    where
        R11, R12 : TOA reflectances for Sentinel-2 bands 11 (1610 nm) and 12 (2190 nm)
        c        : scene-wide slope from a zero-intercept linear regression of
                   R11 on R12 (c = Σ R11·R12 / Σ R12²)

    R_MBSP isolates the methane-induced absorption in band 12 by:
      1) Bringing band 12 onto the radiometric scale of band 11 (multiplying by c)
      2) Differencing and normalising by R11.

    A *negative* R_MBSP indicates that band 12 is darker than expected and is
    therefore consistent with CH₄ absorption.

    Parameters
    ----------
    image  : ee.Image
        Sentinel-2 L1C/L2A image containing bands ‘B11’ and ‘B12’.
    region : ee.Geometry
        Area (ideally plume-free) used to derive the scene-wide slope *c*.

    Returns
    -------
    ee.Image
        Single-band image named ‘R’ holding the pixel-wise MBSP signal.
        The fitted slope *c* is attached as image property ‘slope’.
    """
    # --- 1. Numerator and denominator for the regression slope -----------------
    # c = Σ( R11 * R12 ) / Σ( R12² )
    num_img = image.select("B11").multiply(image.select("B12"))  # R11·R12
    den_img = image.select("B12").multiply(image.select("B12"))  # R12²

    num_sum = num_img.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=region, scale=20, bestEffort=True
    )
    den_sum = den_img.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=region, scale=20, bestEffort=True
    )

    # Convert to ee.Number for further arithmetic
    slope = ee.Number(num_sum.get("B11")).divide(ee.Number(den_sum.get("B12")))

    # --- 2. Per-pixel MBSP field ----------------------------------------------
    mbsp = (
        image.select("B12")
        .multiply(slope)  # c · R12
        .subtract(image.select("B11"))
        .divide(image.select("B11"))
        .rename("R")
    )

    # Embed the slope for traceability
    return mbsp.set({"slope": slope})


## Image Selection

In [ ]:
# Location and date range of interest
lat, lon = 31.6585, 5.9053  # Hassi Messaoud example
start = dt.date(2019, 10, 1)
end = dt.date(2019, 10, 15)

point = ee.Geometry.Point(lon, lat)
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate(str(start), str(end))
    .filterBounds(point)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
    .sort("system:time_start")
    .map(mask_s2_clouds)
)

images = collection.toList(collection.size())
count = images.size().getInfo()
print(f"Found {count} images")

In [ ]:
if count:
    region = point.buffer(1000).bounds()
    m = geemap.Map(center=(lat, lon), zoom=12)
    for i in range(count):
        img = ee.Image(images.get(i))
        date = ee.Date(img.get("system:time_start")).format("YYYY-MM-dd").getInfo()
        r_img = mbsp_fractional_image(img, region)
        m.addLayer(
            r_img,
            {"min": -0.05, "max": 0.05, "palette": ["blue", "white", "red"]},
            f"{date} fractional",
            True,
        )
        rgb = img.select(["B4", "B3", "B2"])
        m.addLayer(rgb, {"min": 0, "max": 0.3}, f"{date} RGB", False)

    styled_pt = ee.FeatureCollection([point]).style(
        **{
            "color": "green",  # outline & fill
            "fillColor": None,  # no fill
            "pointSize": 8,  # pixel radius of the dot
        }
    )
    m.addLayer(styled_pt, {}, "centre pt", True)

In [ ]:
m

The map above contains fractional signal layers for each scene interleaved
with true-color imagery. Fractional layers are visible by default to help
identify methane plumes over time.